In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Read dataset from csv
import pandas as pd
import os
import numpy as np
import time

train_dir = '/content/drive/MyDrive/CapstoneDataset/training'
test_dir = '/content/drive/MyDrive/CapstoneDataset/test'

train_data = []
test_data = []
file_number = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18']

for number in file_number:
  print(train_dir+'/'+number+'_train.csv')
  df = pd.read_csv(train_dir+'/'+number+'_train.csv',header=5)
  df = df.drop('Frame', axis=1)
  df = df.drop('Time (Seconds)', axis=1)
  train_data.append(df)

for number in file_number:
  print(test_dir+'/'+number+'_test.csv')
  df = pd.read_csv(test_dir+'/'+number+'_test.csv',header=5)
  df = df.drop('Frame', axis=1)
  df = df.drop('Time (Seconds)', axis=1)
  test_data.append(df)

/content/drive/MyDrive/CapstoneDataset/training/01_train.csv
/content/drive/MyDrive/CapstoneDataset/training/02_train.csv
/content/drive/MyDrive/CapstoneDataset/training/03_train.csv
/content/drive/MyDrive/CapstoneDataset/training/04_train.csv
/content/drive/MyDrive/CapstoneDataset/training/05_train.csv
/content/drive/MyDrive/CapstoneDataset/training/06_train.csv
/content/drive/MyDrive/CapstoneDataset/training/07_train.csv
/content/drive/MyDrive/CapstoneDataset/training/08_train.csv
/content/drive/MyDrive/CapstoneDataset/training/09_train.csv
/content/drive/MyDrive/CapstoneDataset/training/10_train.csv
/content/drive/MyDrive/CapstoneDataset/training/11_train.csv
/content/drive/MyDrive/CapstoneDataset/training/12_train.csv
/content/drive/MyDrive/CapstoneDataset/training/13_train.csv
/content/drive/MyDrive/CapstoneDataset/training/14_train.csv
/content/drive/MyDrive/CapstoneDataset/training/15_train.csv
/content/drive/MyDrive/CapstoneDataset/training/16_train.csv
/content/drive/MyDrive/C

This part took me over one hour. I exported the final result to csv file. I won't run this part again.

In [ ]:
# Furthest Point Sampling
from os import remove
def read_points(x,y,z):
    points = np.stack([x, y, z], axis=1)
    return points

class FPS:
    def __init__(self, points):
        print('init start')
        self.points = np.unique(points, axis=0)
        print('init finish')

    def get_min_distance(self, a, b):
        distance = []
        for i in range(a.shape[0]):
            dis = np.sum(np.square(a[i] - b), axis=-1)
            distance.append(dis)
        distance = np.stack(distance, axis=-1)
        distance = np.min(distance, axis=-1)
        return np.argmax(distance)
    @staticmethod
    def get_model_corners(model):
        min_x, max_x = np.min(model[:, 0]), np.max(model[:, 0])
        min_y, max_y = np.min(model[:, 1]), np.max(model[:, 1])
        min_z, max_z = np.min(model[:, 2]), np.max(model[:, 2])
        corners_3d = np.array([
            [min_x, min_y, min_z],
            [min_x, min_y, max_z],
            [min_x, max_y, min_z],
            [min_x, max_y, max_z],
            [max_x, min_y, min_z],
            [max_x, min_y, max_z],
            [max_x, max_y, min_z],
            [max_x, max_y, max_z],
        ])
        return corners_3d
    def compute_fps(self, K):
        # compute centre point location
        corner_3d = self.get_model_corners(self.points)
        center_3d = (np.max(corner_3d, 0) + np.min(corner_3d, 0)) / 2
        A = np.array([center_3d])
        B = np.array(self.points)
        t = []
        # looking for k nodes
        for i in range(K):
            # print(i)
            max_id = self.get_min_distance(A, B)
            A = np.append(A, np.array([B[max_id]]), 0)
            B = np.delete(B, max_id, 0)
            t.append(max_id)
        return A, t
        # file.writelines("ply\n")
        # file.writelines("format ascii 1.0\n")
        # file.writelines("element vertex "+str(K+1)+"\n")
        # file.writelines("property float x\n")
        # file.writelines("property float y\n")
        # file.writelines("property float z\n")
        # file.writelines("element face 0\n")
        # file.writelines("property list uchar int vertex_indices\n")
        # file.writelines("end_header\n")

def compute_center_point(dataframe):
  points = pd.DataFrame(data=None,columns=['X','Y','Z'])
  index = 0
  size = dataframe.shape[1]
  for row in dataframe.index:
    x = []
    y = []
    z = []
    for i in range(2,size+1,3):
      x.append(dataframe.loc[row][i-2])
      y.append(dataframe.loc[row][i-1])
      z.append(dataframe.loc[row][i])
    points.loc[index] = [sum(x)/len(x),sum(y)/len(y),sum(z)/len(z)]
    index = index + 1
  return points

# Export sampled data to csv
sampled_train_dir = '/content/drive/MyDrive/CapstoneDataset/sampled_training'
sampled_test_dir = '/content/drive/MyDrive/CapstoneDataset/sampled_test'


count = 1
for df in train_data:
  points = compute_center_point(df)
  print(count)
  f = FPS(read_points(points['X'],points['Y'],points['Z']))

  # Change the sampling number here
  (C,index) = f.compute_fps(100)

  # create a new dataframe has the same column as old df
  data1 = pd.DataFrame(columns = df.columns.to_list())
  for i in index:
    data1 = data1.append(df.loc[i],ignore_index=True)
  data1.to_csv(sampled_train_dir+'/'+str(count)+'.csv', index=False)
  print('finish output')
  count = int(count) + 1

count = 1
for df in test_data:
  points = compute_center_point(df)
  print(count)
  f = FPS(read_points(points['X'],points['Y'],points['Z']))

  # Change the sampling number here
  (C,index) = f.compute_fps(100)

  # create a new dataframe has the same column as old df and export it to csv
  data1 = pd.DataFrame(columns = df.columns.to_list())
  for i in index:
    data1 = data1.append(df.loc[i],ignore_index=True)
  data1.to_csv(sampled_test_dir+'/'+str(count)+'.csv', index=False)
  print('finish output')
  count = int(count) + 1

# count = 1
# for i in range(2,268,3):
#   f = FPS(read_points(df[df.columns[i-2]],df[df.columns[i-1]],df[df.columns[i]]))
#   C = f.compute_fps(100)#Number of sampling points
#   file.writelines("marker"+str(count)+"\n")
#   count = count + 1
#   for j in C:
#     file.writelines(str(float(j[0]))+"\t"+str(float(j[1]))+"\t"+str(float(j[2]))+"\n")

11
init start
init finish
finish output


In [ ]:
# Read sampled data from csv
import pandas as pd
import os
import numpy as np

sampled_train_dir = '/content/drive/MyDrive/CapstoneDataset/sampled_training'
sampled_test_dir = '/content/drive/MyDrive/CapstoneDataset/sampled_test'
sampled_file_number = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18']
sampled_train_data = []
sampled_test_data = []
for number in sampled_file_number:
  print(sampled_train_dir+'/'+number+'.csv')
  df = pd.read_csv(sampled_train_dir+'/'+number+'.csv')
  sampled_train_data.append(df)

for number in sampled_file_number:
  print(sampled_test_dir+'/'+number+'.csv')
  df = pd.read_csv(sampled_test_dir+'/'+number+'.csv')
  sampled_test_data.append(df)

/content/drive/MyDrive/CapstoneDataset/sampled_training/1.csv
/content/drive/MyDrive/CapstoneDataset/sampled_training/2.csv
/content/drive/MyDrive/CapstoneDataset/sampled_training/3.csv
/content/drive/MyDrive/CapstoneDataset/sampled_training/4.csv
/content/drive/MyDrive/CapstoneDataset/sampled_training/5.csv
/content/drive/MyDrive/CapstoneDataset/sampled_training/6.csv
/content/drive/MyDrive/CapstoneDataset/sampled_training/7.csv
/content/drive/MyDrive/CapstoneDataset/sampled_training/8.csv
/content/drive/MyDrive/CapstoneDataset/sampled_training/9.csv
/content/drive/MyDrive/CapstoneDataset/sampled_training/10.csv
/content/drive/MyDrive/CapstoneDataset/sampled_training/11.csv
/content/drive/MyDrive/CapstoneDataset/sampled_training/12.csv
/content/drive/MyDrive/CapstoneDataset/sampled_training/13.csv
/content/drive/MyDrive/CapstoneDataset/sampled_training/14.csv
/content/drive/MyDrive/CapstoneDataset/sampled_training/15.csv
/content/drive/MyDrive/CapstoneDataset/sampled_training/16.csv
/

In [ ]:
sampled_train_data[0]

,X,Y,Z,X.1,Y.1,Z.1,X.2,Y.2,Z.2,X.3,...,Z.85,X.86,Y.86,Z.86,X.87,Y.87,Z.87,X.88,Y.88,Z.88
0,0.422714,0.451719,0.126097,0.413128,0.423567,0.166692,0.390471,0.415988,0.208898,0.387310,...,0.283843,0.605053,0.004932,0.214555,0.591071,0.006551,0.163923,0.542680,0.005298,0.105385
1,0.423924,0.414074,0.114822,0.408149,0.390386,0.160298,0.380680,0.385818,0.201897,0.378510,...,0.282467,0.595580,-0.004032,0.214266,0.583808,-0.005939,0.162342,0.533614,-0.012920,0.102146
2,0.437942,0.424664,0.113667,0.414016,0.401478,0.157248,0.382807,0.399353,0.196396,0.376421,...,0.275294,0.599326,-0.000045,0.206740,0.586417,-0.001408,0.154698,0.536596,-0.006730,0.094737
3,0.426537,0.443731,0.116097,0.410023,0.415812,0.158994,0.382588,0.409248,0.199920,0.381094,...,0.258385,0.593240,0.013162,0.190603,0.579530,0.010217,0.139795,0.527981,0.002303,0.082050
4,0.423517,0.444785,0.122210,0.408916,0.415253,0.164234,0.381776,0.407362,0.205146,0.380275,...,0.262201,0.594109,0.010825,0.193932,0.579826,0.008524,0.143008,0.527106,0.001118,0.086307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.429241,0.484012,0.129133,0.421970,0.453370,0.167380,0.397303,0.446502,0.210846,0.392341,...,0.282623,0.606762,0.023287,0.212654,0.590701,0.025269,0.163304,0.542991,0.024694,0.105979
96,0.440978,0.529347,0.131026,0.429640,0.495460,0.167432,0.404028,0.488174,0.209905,0.398604,...,0.283396,0.601578,0.047479,0.210756,0.584328,0.050669,0.161301,0.534994,0.051023,0.106725
97,0.440933,0.534698,0.136275,0.428967,0.501876,0.173114,0.405865,0.496243,0.216593,0.398977,...,0.281488,0.604263,0.061253,0.209158,0.586188,0.063650,0.160305,0.537254,0.063205,0.106105
98,0.437681,0.517931,0.129753,0.427950,0.484840,0.166381,0.402156,0.476117,0.208732,0.398250,...,0.282481,0.602648,0.037009,0.210173,0.585855,0.039992,0.160525,0.536869,0.040442,0.105427


In [ ]:
arr = pd.DataFrame()
newArr = pd.DataFrame(sampled_train_data[0].iloc[:,0].values)
arr = arr.append(newArr,ignore_index=True)
newArr1 = pd.DataFrame(sampled_train_data[0].iloc[:,3].values)
arr = arr.append(newArr1,ignore_index=True)
cen = arr.sum()/len(arr)


0    0.425454
dtype: float64

In [ ]:
# Normalization

def compute_centroid(pc):
  centroid = []
  x = pd.DataFrame()
  y = pd.DataFrame()
  z = pd.DataFrame()
  for i in range(0,265,3):
    newX = pd.DataFrame(pc.iloc[:,i].values)
    x = x.append(newX,ignore_index=True)
  centroid.append(x.sum()/len(x))
  for j in range(1,266,3):
    newY = pd.DataFrame(pc.iloc[:,j].values)
    y = y.append(newY,ignore_index=True)
  centroid.append(y.sum()/len(y))
  for k in range(2,267,3):
    newZ = pd.DataFrame(pc.iloc[:,k].values)
    z = z.append(newZ,ignore_index=True)
  centroid.append(z.sum()/len(z))
  centroid = pd.DataFrame(centroid)
  return centroid

# centroid = compute_centroid(sampled_train_data[0])

def normalize_point_cloud(pc):
    centroid = compute_centroid(pc) # compute center of point cloud
    centroid = centroid.T
    pc = pc - centroid # put centroid to (0, 0, 0)
    m = np.max(np.sqrt(np.sum(pc ** 2, axis=1))) # compute the length of long axis
    pc_normalized = pc / m # normalize point cloud to (-1,1) according to long axis
    return pc, centroid, m  # centroid: center point, m: length of long axis, centroid and m can be used to compute keypoints


(pc, centroid, length) = normalize_point_cloud(sampled_train_data[0])

In [ ]:
sampled_train_data[0]

,X,Y,Z,X.1,Y.1,Z.1,X.2,Y.2,Z.2,X.3,...,Z.85,X.86,Y.86,Z.86,X.87,Y.87,Z.87,X.88,Y.88,Z.88
0,0.422714,0.451719,0.126097,0.413128,0.423567,0.166692,0.390471,0.415988,0.208898,0.387310,...,0.283843,0.605053,0.004932,0.214555,0.591071,0.006551,0.163923,0.542680,0.005298,0.105385
1,0.423924,0.414074,0.114822,0.408149,0.390386,0.160298,0.380680,0.385818,0.201897,0.378510,...,0.282467,0.595580,-0.004032,0.214266,0.583808,-0.005939,0.162342,0.533614,-0.012920,0.102146
2,0.437942,0.424664,0.113667,0.414016,0.401478,0.157248,0.382807,0.399353,0.196396,0.376421,...,0.275294,0.599326,-0.000045,0.206740,0.586417,-0.001408,0.154698,0.536596,-0.006730,0.094737
3,0.426537,0.443731,0.116097,0.410023,0.415812,0.158994,0.382588,0.409248,0.199920,0.381094,...,0.258385,0.593240,0.013162,0.190603,0.579530,0.010217,0.139795,0.527981,0.002303,0.082050
4,0.423517,0.444785,0.122210,0.408916,0.415253,0.164234,0.381776,0.407362,0.205146,0.380275,...,0.262201,0.594109,0.010825,0.193932,0.579826,0.008524,0.143008,0.527106,0.001118,0.086307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.429241,0.484012,0.129133,0.421970,0.453370,0.167380,0.397303,0.446502,0.210846,0.392341,...,0.282623,0.606762,0.023287,0.212654,0.590701,0.025269,0.163304,0.542991,0.024694,0.105979
96,0.440978,0.529347,0.131026,0.429640,0.495460,0.167432,0.404028,0.488174,0.209905,0.398604,...,0.283396,0.601578,0.047479,0.210756,0.584328,0.050669,0.161301,0.534994,0.051023,0.106725
97,0.440933,0.534698,0.136275,0.428967,0.501876,0.173114,0.405865,0.496243,0.216593,0.398977,...,0.281488,0.604263,0.061253,0.209158,0.586188,0.063650,0.160305,0.537254,0.063205,0.106105
98,0.437681,0.517931,0.129753,0.427950,0.484840,0.166381,0.402156,0.476117,0.208732,0.398250,...,0.282481,0.602648,0.037009,0.210173,0.585855,0.039992,0.160525,0.536869,0.040442,0.105427


In [ ]:
pc

,0,1,2,X,X.1,X.10,X.11,X.12,X.13,X.14,...,Z.80,Z.81,Z.82,Z.83,Z.84,Z.85,Z.86,Z.87,Z.88,Z.9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
centroid

,0
0,0.466186
1,0.203297
2,0.268913


In [ ]:
pc.min().min()

-0.08392678208528297

In [ ]:
centroid

[0     0.471170
 1     0.465548
 2     0.458980
 3     0.465468
 4     0.467833
         ...   
 95    0.473506
 96    0.470966
 97    0.472921
 98    0.470706
 99    0.471477
 Length: 100, dtype: float64, 0     0.192947
 1     0.167383
 2     0.169275
 3     0.183232
 4     0.182719
         ...   
 95    0.218866
 96    0.255892
 97    0.265191
 98    0.244613
 99    0.182725
 Length: 100, dtype: float64, 0     0.272870
 1     0.273504
 2     0.263949
 3     0.260059
 4     0.264746
         ...   
 95    0.272803
 96    0.273375
 97    0.276635
 98    0.271823
 99    0.258827
 Length: 100, dtype: float64]

In [ ]:
length

0.6837376410034263